<a href="https://colab.research.google.com/github/RicardoPoleo/DeepLearning_FactChecker/blob/main/notebooks/Agents/Agent_1_Model_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading, infering and evaluating the models for benchmarking


In [ ]:
from transformers import pipeline
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from google.colab import data_table

# Define the models and prompts
model_names = [
    "dmis-lab/biobert-base-cased-v1.2",
    "dmis-lab/biobert-v1.1",
    "dmis-lab/biobert-base-cased-v1.1",
    "pritamdeka/BioBert-PubMed200kRCT",
    "microsoft/deberta-v3-base",
    "Clinical-AI-Apollo/Medical-NER"
]

prompts = [
  'taking vitamin c prevents the common cold.',
  'ibuprofen is effective for reducing fever.'
]

expected_entities = {
    'taking vitamin c prevents the common cold.': ["vitamin c", "common cold"],
    'ibuprofen is effective for reducing fever.': ['ibuprofen', 'fever'],
}

def load_model_for_token_classification(model_name, strategy='simple'):
    return pipeline("token-classification", model=model_name, aggregation_strategy=strategy)

def load_models(model_names, strategy='simple'):
    models_dict = {}
    for model_name in model_names:
        print(f"Loading model {model_name} with strategy {strategy}")
        model = load_model_for_token_classification(model_name, strategy=strategy)
        models_dict[model_name] = model
    print(f"Loaded {len(models_dict)} models with strategy {strategy}")
    return models_dict

def evaluate_models(models_dict, prompts):
    results = {}
    for model_name, model in models_dict.items():
        print(f"Evaluating model {model_name}")
        results[model_name] = {}
        for prompt in prompts:
            result = model(prompt)
            results[model_name][prompt] = result
    return results

def extract_entities(results):
    extracted_entities = {}
    for model_name, model_results in results.items():
        extracted_entities[model_name] = {}
        for prompt, entities in model_results.items():
            extracted = [(entity['entity_group'].lower(), entity['word'].lower()) for entity in entities]
            extracted_entities[model_name][prompt] = extracted
            # Debugging: Print extracted entities
            print(f"Model: {model_name}, Prompt: {prompt}, Extracted: {extracted}")
    return extracted_entities

def calculate_metrics(extracted_entities, expected_entities, strategy):
    metrics = {}
    for model_name, model_entities in extracted_entities.items():
        y_true = []
        y_pred = []
        for prompt, expected in expected_entities.items():
            pred_entities = model_entities.get(prompt, [])
            true_labels = expected  # Use expected directly as list of entities
            pred_labels = [entity for _, entity in pred_entities]

            # Debugging: Print true and predicted labels
            print(f"Model: {model_name}, Strategy: {strategy}, Prompt: {prompt}, True Labels: {true_labels}, Pred Labels: {pred_labels}")

            # Ensure lengths match
            min_length = min(len(true_labels), len(pred_labels))
            true_labels = true_labels[:min_length]
            pred_labels = pred_labels[:min_length]

            y_true.extend(true_labels)
            y_pred.extend(pred_labels)

        precision = precision_score(y_true, y_pred, average='macro', zero_division=1)
        recall = recall_score(y_true, y_pred, average='macro', zero_division=1)
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)

        metrics[model_name] = {'precision': precision, 'recall': recall, 'f1': f1}

    return metrics

def metrics_to_dataframe(metrics_by_strategy):
    rows = []
    for strategy, metrics in metrics_by_strategy.items():
        for model_name, metric_values in metrics.items():
            rows.append({
                "Strategy": strategy,
                "Model": model_name,
                "Precision": metric_values['precision'],
                "Recall": metric_values['recall'],
                "F1 Score": metric_values['f1']
            })
    df = pd.DataFrame(rows)
    return df

# Define aggregation strategies
SIMPLE = "simple"
FIRST = "first"
AVERAGE = "average"
MAX = "max"

# Store models and results for each strategy
models_dict_by_strategy = {
    SIMPLE: {},
    FIRST: {},
    AVERAGE: {},
    MAX: {}
}

# Load models for each strategy
for strategy in [SIMPLE, FIRST, AVERAGE, MAX]:
    models_dict_by_strategy[strategy] = load_models(model_names, strategy=strategy)

# Evaluate models and calculate metrics for each strategy
results_by_strategy = {}
metrics_by_strategy = {}

for strategy, models_dict in models_dict_by_strategy.items():
    print(f"Evaluating models with strategy {strategy}")
    results = evaluate_models(models_dict, prompts)
    extracted_entities = extract_entities(results)
    metrics = calculate_metrics(extracted_entities, expected_entities, strategy)
    results_by_strategy[strategy] = results
    metrics_by_strategy[strategy] = metrics

# Convert metrics to a dataframe and display
df_metrics = metrics_to_dataframe(metrics_by_strategy)

# Display the dataframe as an interactive table
data_table.DataTable(df_metrics, include_index=False, num_rows_per_page=10)


Loading model dmis-lab/biobert-base-cased-v1.2 with strategy simple


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model dmis-lab/biobert-v1.1 with strategy simple


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model dmis-lab/biobert-base-cased-v1.1 with strategy simple


...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating model pritamdeka/BioBert-PubMed200kRCT
Evaluating model microsoft/deberta-v3-base


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating model Clinical-AI-Apollo/Medical-NER
Model: dmis-lab/biobert-base-cased-v1.2, Prompt: taking vitamin c prevents the common cold., Extracted: [('label_1', 'taking vitamin'), ('label_0', 'c'), ('label_1', 'prevents'), ('label_0', 'the common'), ('label_1', 'cold'), ('label_0', '.')]
Model: dmis-lab/biobert-base-cased-v1.2, Prompt: ibuprofen is effective for reducing fever., Extracted: [('label_1', 'ibuprofen'), ('label_0', 'is'), ('label_1', 'effective for reducing'), ('label_0', 'fever.')]
Model: dmis-lab/biobert-v1.1, Prompt: taking vitamin c prevents the common cold., Extracted: [('label_0', 'taking vitamin'), ('label_1', 'c'), ('label_0', 'prevents the common cold.')]
Model: dmis-lab/biobert-v1.1, Prompt: ibuprofen is effective for reducing fever., Extracted: [('label_0', 'ibuprofen is effective'), ('label_1', 'for'), ('label_0', 'reducing fever.')]
Model: dmis-lab/biobert-base-cased-v1.1, Prompt: taking vitamin c prevents the common cold., Extracted: [('label_0', 'taking 

,Strategy,Model,Precision,Recall,F1 Score
0,simple,dmis-lab/biobert-base-cased-v1.2,0.500000,0.500000,0.000000
1,simple,dmis-lab/biobert-v1.1,0.500000,0.500000,0.000000
2,simple,dmis-lab/biobert-base-cased-v1.1,0.500000,0.500000,0.000000
3,simple,pritamdeka/BioBert-PubMed200kRCT,0.500000,0.500000,0.000000
4,simple,microsoft/deberta-v3-base,0.500000,0.500000,0.000000
5,simple,Clinical-AI-Apollo/Medical-NER,0.666667,0.666667,0.333333
6,first,dmis-lab/biobert-base-cased-v1.2,0.500000,0.500000,0.000000
7,first,dmis-lab/biobert-v1.1,0.500000,0.500000,0.000000
8,first,dmis-lab/biobert-base-cased-v1.1,0.600000,0.600000,0.200000
9,first,pritamdeka/BioBert-PubMed200kRCT,0.500000,0.500000,0.000000


# Loading and calculating basic metrics

In [ ]:
import pandas as pd
from google.colab import data_table
df_metrics = metrics_to_dataframe(metrics_by_strategy)
data_table.DataTable(df_metrics, include_index=False, num_rows_per_page=10)

,Strategy,Model,Precision,Recall,F1 Score
0,simple,dmis-lab/biobert-base-cased-v1.2,0.600000,0.400000,0.0
1,simple,dmis-lab/biobert-v1.1,0.600000,0.400000,0.0
2,simple,dmis-lab/biobert-base-cased-v1.1,0.666667,0.333333,0.0
3,simple,pritamdeka/BioBert-PubMed200kRCT,0.666667,0.333333,0.0
4,simple,microsoft/deberta-v3-base,0.666667,0.333333,0.0
5,simple,Clinical-AI-Apollo/Medical-NER,0.500000,0.500000,0.0
6,first,dmis-lab/biobert-base-cased-v1.2,0.600000,0.400000,0.0
7,first,dmis-lab/biobert-v1.1,0.666667,0.333333,0.0
8,first,dmis-lab/biobert-base-cased-v1.1,0.666667,0.333333,0.0
9,first,pritamdeka/BioBert-PubMed200kRCT,0.666667,0.333333,0.0


from matplotlib import pyplot as plt
_df_0['Precision'].plot(kind='hist', bins=20, title='Precision')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Recall'].plot(kind='hist', bins=20, title='Recall')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Strategy').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Model').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='Precision', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Strategy'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Recall', y='Strategy', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Recall', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)